# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143
The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! yo

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab=tuple(set(text))
    int_to_vocab=dict(enumerate(vocab))
    vocab_to_int={ch:ii for ii,ch in int_to_vocab.items()}
    encoded = np.array([vocab_to_int[ch] for ch in text])
    # return tuple
    return (vocab_to_int, int_to_vocab)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [4]:
#Test the function
create_lookup_tables(lines)

({'': 0,
  "elaine: your hair. it's so thick and lustrous. i mean, it...it was. ": 1,
  "george: (agitated) he's lying, sir! i'm tired of all your macho head games! ": 2,
  "elaine: y'know i can think of at *least* six known offensive odours that i would *rather* smell than what's livin' in your car. ": 3,
  "george: i don't even know peterman. how the hell am i gonna relax? i'm gonna have to be on all night. i don't like being on, jerry, i would much rather be off. ": 4,
  'elaine: it is not necessary. ': 5,
  'katie: and, it is already in the school paper. they cancelled rick james. ': 6,
  "mickey: oh alright, alright. i'll be right over. ": 7,
  'kramer: tell me about it. ': 8,
  "[movie theater: schindler's list] ": 9,
  "russell: pfeiffer's. ": 10,
  "morty: what are you talking about? i'm the one who should have called the police. they stole my wallet. ": 11,
  "sal bass: he's a member of our health club. isn't he? ": 12,
  'jerry: "oh! where\'s jennifer today?" ': 13,
  'estell

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    
    from collections import Counter
    word_counts = Counter(lines)
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
    vocab_to_int = dict()
    int_to_vocab = dict()
    
    for i, word in enumerate(sorted_words):
        vocab_to_int[word] = i
        int_to_vocab[i] = word
    return {
        '.'  : '||period||',
        ','  : '||comma||',
        '"'  : '||quotationmark||',
        ';'  : '||semicolon||',
        '!'  : '||exclamationmark||',
        '?'  : '||questionmark||',
        '('  : '||leftparentheses',
        ')'  : '||rightparentheses',
        '\n' : '||return||',
        '-'  : '||dash||'
   }  
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [9]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    n_batches = len(words)//sequence_length
    
    # Complete batches
    words = words[:n_batches*sequence_length]
    words = np.array(words)
    words = words.reshape(n_batches, sequence_length)
    feature_tensors = torch.Tensor(words)
    
    targets = []
    for idx in range(0, n_batches):
        try:
            targets.append(words[idx+1][0])
            
        except IndexError:
            targets.append(0)
            
    target_tensors = torch.Tensor(np.array(targets))
    
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)   

    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [10]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24.],
        [25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34.],
        [35., 36., 37., 38., 39.],
        [40., 41., 42., 43., 44.],
        [45., 46., 47., 48., 49.]])

torch.Size([10])
tensor([ 5., 10., 15., 20., 25., 30., 35., 40., 45.,  0.])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
       
     
        super(RNN, self).__init__()      
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=0.5, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_size) 
       
        
        # define model layers
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        output = self.fc(lstm_out)
        
        # reshape to batch_size
        output = output.view(batch_size,-1,self.output_size)
        out = output[:, -1]
                
        # return the neural network output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())       
        return hidden


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
       if (train_on_gpu):
            inp, target = inp.cuda(), target.cuda()

            # Detach passed in hidden_state from it's history.
            hidden = tuple([each.data for each in hidden])

            # Clear gradients
            rnn.zero_grad()

            # Perform forward pass
            output, hidden = rnn(inp, hidden)

            loss = criterion(output, target.long())
            loss.backward()

            # Clip to prevent exploding gradients? 
            # nn.utils.clip_grad_norm_(rnn.parameters(),5)
            optimizer.step()

            # return the loss over a batch and the hidden state produced by our model
            return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            #if batch_i % show_every_n_batches == 0:
            print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(epoch_i, n_epochs, np.average(batch_losses)))
            batch_losses = []



    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [14]:
# Data params
# Sequence Length
sequence_length = 32  # of words in a sequence
# Batch Size
batch_size = 64

# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 128
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 9.96628475189209

Epoch:    1/10    Loss: 9.940866470336914

Epoch:    1/10    Loss: 9.899784088134766

Epoch:    1/10    Loss: 9.868261337280273

Epoch:    1/10    Loss: 9.564094543457031

Epoch:    1/10    Loss: 9.372947692871094

Epoch:    1/10    Loss: 7.961690425872803

Epoch:    1/10    Loss: 7.615900039672852

Epoch:    1/10    Loss: 6.86991024017334

Epoch:    1/10    Loss: 6.71161413192749

Epoch:    1/10    Loss: 7.3555803298950195

Epoch:    1/10    Loss: 7.298036575317383

Epoch:    1/10    Loss: 7.2166266441345215

Epoch:    1/10    Loss: 6.18456506729126

Epoch:    1/10    Loss: 6.747117042541504

Epoch:    1/10    Loss: 7.415852069854736

Epoch:    1/10    Loss: 6.064113140106201

Epoch:    1/10    Loss: 6.535752296447754

Epoch:    1/10    Loss: 6.161320209503174

Epoch:    1/10    Loss: 6.765669822692871

Epoch:    1/10    Loss: 6.511560440063477

Epoch:    1/10    Loss: 6.457286357879639

Epoch:    1/10    Loss: 6.62

Epoch:    1/10    Loss: 5.776693820953369

Epoch:    1/10    Loss: 5.3635993003845215

Epoch:    1/10    Loss: 5.495243549346924

Epoch:    1/10    Loss: 5.605137825012207

Epoch:    1/10    Loss: 4.804430961608887

Epoch:    1/10    Loss: 5.648684978485107

Epoch:    1/10    Loss: 5.856825351715088

Epoch:    1/10    Loss: 6.586742401123047

Epoch:    1/10    Loss: 5.5565972328186035

Epoch:    1/10    Loss: 5.95997428894043

Epoch:    1/10    Loss: 6.187102794647217

Epoch:    1/10    Loss: 5.820664405822754

Epoch:    1/10    Loss: 5.932291030883789

Epoch:    1/10    Loss: 5.522522449493408

Epoch:    1/10    Loss: 5.556591987609863

Epoch:    1/10    Loss: 5.338473320007324

Epoch:    1/10    Loss: 5.445347309112549

Epoch:    1/10    Loss: 4.894041538238525

Epoch:    1/10    Loss: 5.8965277671813965

Epoch:    1/10    Loss: 5.071646213531494

Epoch:    1/10    Loss: 5.270015239715576

Epoch:    1/10    Loss: 5.635853290557861

Epoch:    1/10    Loss: 6.4040727615356445

Epoch:  

Epoch:    1/10    Loss: 4.888939380645752

Epoch:    1/10    Loss: 4.96409273147583

Epoch:    1/10    Loss: 5.5009765625

Epoch:    1/10    Loss: 5.902310848236084

Epoch:    1/10    Loss: 4.767575740814209

Epoch:    1/10    Loss: 5.534261226654053

Epoch:    1/10    Loss: 4.709120750427246

Epoch:    1/10    Loss: 5.221793174743652

Epoch:    1/10    Loss: 4.944847106933594

Epoch:    1/10    Loss: 5.438757419586182

Epoch:    1/10    Loss: 5.697683334350586

Epoch:    1/10    Loss: 5.2058539390563965

Epoch:    1/10    Loss: 4.447365760803223

Epoch:    1/10    Loss: 5.362141132354736

Epoch:    1/10    Loss: 5.772316932678223

Epoch:    1/10    Loss: 4.810460090637207

Epoch:    1/10    Loss: 5.131880760192871

Epoch:    1/10    Loss: 5.198602199554443

Epoch:    1/10    Loss: 5.648744583129883

Epoch:    1/10    Loss: 5.2976908683776855

Epoch:    1/10    Loss: 5.607816696166992

Epoch:    1/10    Loss: 5.134333610534668

Epoch:    1/10    Loss: 5.155048847198486

Epoch:    1/10 

Epoch:    2/10    Loss: 4.0472540855407715

Epoch:    2/10    Loss: 5.1117048263549805

Epoch:    2/10    Loss: 4.074642181396484

Epoch:    2/10    Loss: 4.809853553771973

Epoch:    2/10    Loss: 4.56231164932251

Epoch:    2/10    Loss: 4.307973384857178

Epoch:    2/10    Loss: 4.702823638916016

Epoch:    2/10    Loss: 4.573821067810059

Epoch:    2/10    Loss: 5.295618534088135

Epoch:    2/10    Loss: 4.455352306365967

Epoch:    2/10    Loss: 4.61738920211792

Epoch:    2/10    Loss: 4.890681266784668

Epoch:    2/10    Loss: 4.829899787902832

Epoch:    2/10    Loss: 4.467574119567871

Epoch:    2/10    Loss: 4.4319305419921875

Epoch:    2/10    Loss: 4.791907787322998

Epoch:    2/10    Loss: 4.669086933135986

Epoch:    2/10    Loss: 5.06855583190918

Epoch:    2/10    Loss: 4.914313316345215

Epoch:    2/10    Loss: 4.288352966308594

Epoch:    2/10    Loss: 5.538208484649658

Epoch:    2/10    Loss: 4.527510643005371

Epoch:    2/10    Loss: 5.3059844970703125

Epoch:    

Epoch:    2/10    Loss: 5.167539596557617

Epoch:    2/10    Loss: 4.873900890350342

Epoch:    2/10    Loss: 5.126552581787109

Epoch:    2/10    Loss: 4.123300552368164

Epoch:    2/10    Loss: 4.351076126098633

Epoch:    2/10    Loss: 4.2932209968566895

Epoch:    2/10    Loss: 5.270920753479004

Epoch:    2/10    Loss: 5.4999494552612305

Epoch:    2/10    Loss: 4.659820556640625

Epoch:    2/10    Loss: 4.221273422241211

Epoch:    2/10    Loss: 4.760315895080566

Epoch:    2/10    Loss: 4.721426010131836

Epoch:    2/10    Loss: 4.2527313232421875

Epoch:    2/10    Loss: 4.868250370025635

Epoch:    2/10    Loss: 4.821719169616699

Epoch:    2/10    Loss: 4.365938663482666

Epoch:    2/10    Loss: 4.629260540008545

Epoch:    2/10    Loss: 5.179280757904053

Epoch:    2/10    Loss: 4.178362846374512

Epoch:    2/10    Loss: 5.280031681060791

Epoch:    2/10    Loss: 4.933816432952881

Epoch:    2/10    Loss: 4.041530132293701

Epoch:    2/10    Loss: 5.100112438201904

Epoch:  

...

Epoch:    8/10    Loss: 2.5527610778808594

Epoch:    8/10    Loss: 3.959604501724243

Epoch:    8/10    Loss: 3.805612564086914

Epoch:    8/10    Loss: 3.6622297763824463

Epoch:    8/10    Loss: 2.9666621685028076

Epoch:    8/10    Loss: 3.3222320079803467

Epoch:    8/10    Loss: 3.2251977920532227

Epoch:    8/10    Loss: 2.9920616149902344

Epoch:    8/10    Loss: 3.4082372188568115

Epoch:    8/10    Loss: 3.2628986835479736

Epoch:    8/10    Loss: 3.3239688873291016

Epoch:    8/10    Loss: 3.264254570007324

Epoch:    8/10    Loss: 2.940685272216797

Epoch:    8/10    Loss: 3.011773109436035

Epoch:    8/10    Loss: 3.3636038303375244

Epoch:    8/10    Loss: 3.5609183311462402

Epoch:    8/10    Loss: 3.0171291828155518

Epoch:    8/10    Loss: 3.761061429977417

Epoch:    8/10    Loss: 3.3399837017059326

Epoch:    8/10    Loss: 3.443450927734375

Epoch:    8/10    Loss: 3.9760406017303467

Epoch:    8/10    Loss: 3.096611499786377

Epoch:    8/10    Loss: 3.42015504837036

Epoch:    9/10    Loss: 3.6485695838928223

Epoch:    9/10    Loss: 3.531252384185791

Epoch:    9/10    Loss: 3.1879637241363525

Epoch:    9/10    Loss: 3.468090772628784

Epoch:    9/10    Loss: 3.5014069080352783

Epoch:    9/10    Loss: 3.1627376079559326

Epoch:    9/10    Loss: 3.0256495475769043

Epoch:    9/10    Loss: 3.783264636993408

Epoch:    9/10    Loss: 2.9326119422912598

Epoch:    9/10    Loss: 3.207808494567871

Epoch:    9/10    Loss: 3.497070789337158

Epoch:    9/10    Loss: 2.986701250076294

Epoch:    9/10    Loss: 2.7727386951446533

Epoch:    9/10    Loss: 3.461139678955078

Epoch:    9/10    Loss: 3.7431094646453857

Epoch:    9/10    Loss: 4.176585674285889

Epoch:    9/10    Loss: 3.449401378631592

Epoch:    9/10    Loss: 3.4133639335632324

Epoch:    9/10    Loss: 3.0375828742980957

Epoch:    9/10    Loss: 3.5648603439331055

Epoch:    9/10    Loss: 3.330824136734009

Epoch:    9/10    Loss: 3.4099109172821045

Epoch:    9/10    Loss: 3.616318702697754


Epoch:    9/10    Loss: 3.4402287006378174

Epoch:    9/10    Loss: 2.785649061203003

Epoch:    9/10    Loss: 3.076388120651245

Epoch:    9/10    Loss: 3.4843826293945312

Epoch:    9/10    Loss: 2.9255659580230713

Epoch:    9/10    Loss: 3.3356432914733887

Epoch:    9/10    Loss: 3.4829211235046387

Epoch:    9/10    Loss: 3.451742172241211

Epoch:    9/10    Loss: 3.031914234161377

Epoch:    9/10    Loss: 3.4604451656341553

Epoch:    9/10    Loss: 3.000154733657837

Epoch:    9/10    Loss: 3.0691280364990234

Epoch:    9/10    Loss: 3.4321062564849854

Epoch:    9/10    Loss: 3.1384077072143555

Epoch:    9/10    Loss: 2.769711494445801

Epoch:    9/10    Loss: 3.345123052597046

Epoch:    9/10    Loss: 3.160214900970459

Epoch:    9/10    Loss: 3.2726035118103027

Epoch:    9/10    Loss: 3.234776496887207

Epoch:    9/10    Loss: 2.757157802581787

Epoch:    9/10    Loss: 3.1117379665374756

Epoch:    9/10    Loss: 3.526883125305176

Epoch:    9/10    Loss: 2.9611032009124756


Epoch:   10/10    Loss: 3.11830472946167

Epoch:   10/10    Loss: 3.165733575820923

Epoch:   10/10    Loss: 3.0100648403167725

Epoch:   10/10    Loss: 2.9797701835632324

Epoch:   10/10    Loss: 3.3470664024353027

Epoch:   10/10    Loss: 3.186392307281494

Epoch:   10/10    Loss: 3.2125003337860107

Epoch:   10/10    Loss: 2.64729642868042

Epoch:   10/10    Loss: 3.0842819213867188

Epoch:   10/10    Loss: 3.5209808349609375

Epoch:   10/10    Loss: 3.0194997787475586

Epoch:   10/10    Loss: 3.098245620727539

Epoch:   10/10    Loss: 3.4349167346954346

Epoch:   10/10    Loss: 3.2854831218719482

Epoch:   10/10    Loss: 3.3719582557678223

Epoch:   10/10    Loss: 3.0620052814483643

Epoch:   10/10    Loss: 3.218393325805664

Epoch:   10/10    Loss: 3.1341664791107178

Epoch:   10/10    Loss: 2.8850560188293457

Epoch:   10/10    Loss: 3.000385046005249

Epoch:   10/10    Loss: 2.9410812854766846

Epoch:   10/10    Loss: 3.2664833068847656

Epoch:   10/10    Loss: 2.6442711353302



Epoch:   10/10    Loss: 3.1606788635253906

Epoch:   10/10    Loss: 2.269169330596924

Epoch:   10/10    Loss: 2.9847328662872314

Epoch:   10/10    Loss: 3.117649793624878

Epoch:   10/10    Loss: 2.3388161659240723

Epoch:   10/10    Loss: 2.965794324874878

Epoch:   10/10    Loss: 2.9531850814819336

Epoch:   10/10    Loss: 3.447871208190918

Epoch:   10/10    Loss: 2.851473093032837

Epoch:   10/10    Loss: 2.7711031436920166

Epoch:   10/10    Loss: 2.822049617767334

Epoch:   10/10    Loss: 3.1926894187927246

Epoch:   10/10    Loss: 2.975689172744751

Epoch:   10/10    Loss: 2.8979530334472656

Epoch:   10/10    Loss: 3.447122573852539

Epoch:   10/10    Loss: 3.110517978668213

Epoch:   10/10    Loss: 2.9561774730682373

Epoch:   10/10    Loss: 2.9757516384124756

Epoch:   10/10    Loss: 2.7299067974090576

Epoch:   10/10    Loss: 3.02787446975708

Epoch:   10/10    Loss: 2.6286771297454834

Epoch:   10/10    Loss: 2.808791399002075

Epoch:   10/10    Loss: 3.3011114597320557



### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

#### **Answer:** Tried to follow best bractice mentioned by Jay in the course for first try intiating learning rate and decrease it or increase it based on the model performance.
Also selecting the batch size of 32 to be in the recommended range 32,64,128,256 , achieved 2.42 Loss with 10 epochs

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [22]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

kramer: code) oh ho, i'm not skittles sophisticated via?

elaine: oh, yeah, i don't caer you the suitcases?

jerry: i know! stacy not a feeling note

elaine: i wish this sophisticated funny!

jerry: i don't understand what? i'm just change a le hat in the robbery] socializing for his rickshaw code

jerry: i don't know, you know, i sometimes about this?

jerry: you should know, i'm not wanted to get jiggy the deputies, jesus!

jerry: well, i think i spend have it about the city?

george: i am have given sophisticated?

jerry:(nervous laughs) you'd not fancy.

jerry:(shocked) : he wants to the heat.

george: well, everybody here what?

jerry: i am clear

elaine: no! i just wanted a rickshaw.

kramer:(nervous head on a huge bounce talkie) : you should clear about the bumper?

jerry: i know, you could know what sophisticated that?

george: well, i'm not fancy?

kramer:(nervous head) oh ho, you think this is the movies!

elaine: i know what you know you were to get a maid?

george: oh, ever

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [19]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.